## Cryptography
### Task № 1 - Chiffre de Vigenère
### Ivan Rybin - ITMO JB SE MA 2021

In [68]:
import math
from tqdm import tqdm 

In [69]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [70]:
def gen_all_offsets(alphabet: str):
    offsets = []
    alen = len(alphabet)
    chars = [c for c in alphabet]
    for i in range(0, alen):
        offsets.append([chars[j % alen] for j in range(i, alen + i)])
    return offsets

def print_offsets(offsets: list):
    i = 0
    for offset in offsets:
        print("".join(offset) + " - " + str(i))
        i += 1

In [71]:
print_offsets(gen_all_offsets(alphabet))

abcdefghijklmnopqrstuvwxyz - 0
bcdefghijklmnopqrstuvwxyza - 1
cdefghijklmnopqrstuvwxyzab - 2
defghijklmnopqrstuvwxyzabc - 3
efghijklmnopqrstuvwxyzabcd - 4
fghijklmnopqrstuvwxyzabcde - 5
ghijklmnopqrstuvwxyzabcdef - 6
hijklmnopqrstuvwxyzabcdefg - 7
ijklmnopqrstuvwxyzabcdefgh - 8
jklmnopqrstuvwxyzabcdefghi - 9
klmnopqrstuvwxyzabcdefghij - 10
lmnopqrstuvwxyzabcdefghijk - 11
mnopqrstuvwxyzabcdefghijkl - 12
nopqrstuvwxyzabcdefghijklm - 13
opqrstuvwxyzabcdefghijklmn - 14
pqrstuvwxyzabcdefghijklmno - 15
qrstuvwxyzabcdefghijklmnop - 16
rstuvwxyzabcdefghijklmnopq - 17
stuvwxyzabcdefghijklmnopqr - 18
tuvwxyzabcdefghijklmnopqrs - 19
uvwxyzabcdefghijklmnopqrst - 20
vwxyzabcdefghijklmnopqrstu - 21
wxyzabcdefghijklmnopqrstuv - 22
xyzabcdefghijklmnopqrstuvw - 23
yzabcdefghijklmnopqrstuvwx - 24
zabcdefghijklmnopqrstuvwxy - 25


In [72]:
def repeat_key(key, text):
    tlen = len(text)
    klen = len(key)
    k = tlen // klen
    return key * k + key[0:tlen % klen]

In [73]:
for i in range(0, 10):
    print(f"{i}: {repeat_key('key', '1' * i)}")

0: 
1: k
2: ke
3: key
4: keyk
5: keyke
6: keykey
7: keykeyk
8: keykeyke
9: keykeykey


In [74]:
def encode_vigenere(key, text, alphabet):
    alpha_ids = dict()
    for i in range(0, len(alphabet)):
        alpha_ids[alphabet[i]] = i
        
    alpha_offsets = gen_all_offsets(alphabet)
    repeated_key = repeat_key(key, text)
    encrypted_msg = []
    k = 0
    for c in repeated_key:
        i = alpha_ids[c]
        j = alpha_ids[text[k]]
        encrypted_msg.append(alpha_offsets[i][j])
        k += 1
    return "".join(encrypted_msg)


def decode_vigenere(key, encoded_text, alphabet):
    alpha_ids = dict()
    for i in range(0, len(alphabet)):
        alpha_ids[alphabet[i]] = i
    
    alpha_offsets = gen_all_offsets(alphabet)
    repeated_key = repeat_key(key, encoded_text)
    decrypted_msg = []
    k = 0
    for c in repeated_key:
        i = alpha_ids[c]
        a = encoded_text[k]
        for j in range(0, len(alphabet)):
            if a == alpha_offsets[i][j]:
                decrypted_msg.append(alphabet[j])
        k += 1
        
    return "".join(decrypted_msg)

In [75]:
encode_vigenere("CRYPTO", "HELLOWORLD", alphabet.upper())

'JVJAHKQIJS'

In [76]:
decode_vigenere("CRYPTO", "JVJAHKQIJS", alphabet.upper())

'HELLOWORLD'

In [77]:
def gen_ngrams(text, n=3):
    return [text[i:i+n] for i in range(0, len(text) - 3 + 1)]


def count_ngrams(ngrams):
    ngrams_cnt = dict()
    for ngram in ngrams:
        if ngram not in ngrams_cnt:
            ngrams_cnt[ngram] = 0
        ngrams_cnt[ngram] += 1
    return {item[0]: item[1] for item in sorted(ngrams_cnt.items(), reverse=True, key=lambda item: item[1])}


def gen_ngrams_positions(text, n = 3):
    ngrams = count_ngrams(gen_ngrams(text, n))
    ngrams_pos = dict()
    for ngram, cnt in ngrams.items():
        for i in range(0, len(text)):
            if text[i:i + n] == ngram:
                if ngram not in ngrams_pos:
                    ngrams_pos[ngram] = []
                ngrams_pos[ngram].append(i)
    return ngrams_pos


def gen_ngrams_deltas(text, n = 3):
    ngrams_pos = gen_ngrams_positions(text, n)
    ngrams_deltas = dict()
    for (ngram, poss) in ngrams_pos.items():
        deltas = []
        if (len(poss) >= 2):
            for i in range(1, len(poss)):
                deltas.append(poss[i] - poss[i - 1])
        if len(deltas) > 1:
            ngrams_deltas[ngram] = deltas
    return ngrams_deltas


def get_gcdc_ngrams(text, n = 3):
    ngrams_deltas = gen_ngrams_deltas(text, n)
    ngrams_gcds = dict()
    for (ngram, deltas) in ngrams_deltas.items():
        if len(deltas) > 1:
            gcd = deltas[0]
            for k in deltas:
                gcd = math.gcd(gcd, k)
            ngrams_gcds[ngram] = gcd
    return ngrams_gcds

def get_keys_length(text, n=3):
    return set(get_gcdc_ngrams(text, n).values())


def get_top_key_length(text, n=3):
    top_keys = dict()
    for k, v in get_gcdc_ngrams(text, n).items():
        if v not in top_keys:
            top_keys[v] = 0
        top_keys[v] += 1
    return [item for item in sorted(top_keys.items(), reverse=True, key=lambda item: item[1])]

In [78]:
gen_ngrams("privet")

['pri', 'riv', 'ive', 'vet']

In [79]:
count_ngrams(gen_ngrams("allallallaaaallabbbbbbbbbbbbbbbbballallallallaaaallabbbbbbbbbbbbbbbbballaa"))

{'bbb': 30,
 'all': 10,
 'lla': 10,
 'lal': 5,
 'aaa': 4,
 'laa': 3,
 'aal': 2,
 'lab': 2,
 'abb': 2,
 'bba': 2,
 'bal': 2}

In [80]:
gen_ngrams_positions("allallallaaaallaballallallallaaaallabbbbbbbbbbbbbbbbballaa")

{'bbb': [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50],
 'all': [0, 3, 6, 12, 17, 20, 23, 26, 32, 53],
 'lla': [1, 4, 7, 13, 18, 21, 24, 27, 33, 54],
 'lal': [2, 5, 19, 22, 25],
 'aaa': [9, 10, 29, 30],
 'laa': [8, 28, 55],
 'aal': [11, 31],
 'lab': [14, 34],
 'bal': [16, 52],
 'aba': [15],
 'abb': [35],
 'bba': [51]}

In [81]:
gen_ngrams_deltas("allallallaaaallaballallallallaaaallabbbbbbbbbbbbbbbbballaa")

{'bbb': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'all': [3, 3, 6, 5, 3, 3, 3, 6, 21],
 'lla': [3, 3, 6, 5, 3, 3, 3, 6, 21],
 'lal': [3, 14, 3, 3],
 'aaa': [1, 19, 1],
 'laa': [20, 27]}

In [82]:
get_gcdc_ngrams("allallallaaaallabbbbbbbbbbbbbbbbballallallallaaaallabbbbbbbbbbbbbbbbballaa")

{'bbb': 1, 'all': 3, 'lla': 3, 'lal': 3, 'aaa': 1, 'laa': 9}

In [83]:
get_keys_length("allallallaaaallabbbbbbbbbbbbbbbbballallallallaaaallabbbbbbbbbbbbbbbbballaa")

{1, 3, 9}

In [84]:
get_top_key_length("allallallaaaallabbbbbbbbbbbbbbbbballallallallaaaallabbbbbbbbbbbbbbbbballaa")

[(3, 3), (1, 2), (9, 1)]

In [85]:
def build_alpha_stats(file_path: str, alphabet):
    chars = dict()
    with open(file_path, 'r') as f:
        for line in tqdm(f):
            for c in line.lower():
                if c not in chars:
                    chars[c] = 0
                chars[c] += 1
    total = 0
    chars_alpha = dict()
    for c, cnt in chars.items():
        if c in alphabet:
            total += cnt
            
    for c, cnt in chars.items():
        if c in alphabet:
            chars_alpha[c] = cnt / total
            
    return [item for item in sorted(chars_alpha.items(), reverse=True, key=lambda item: item[1])]

In [165]:
# alpha_analysis = build_alpha_stats('enwiki-20210120-pages-articles-multistream1.xml-p1p41242', alphabet)

alpha_analysis = [
    ('a', 0.082),
    ('b', 0.015),
    ('c', 0.028),
    ('d', 0.043),
    ('e', 0.127),
    ('f', 0.022),
    ('g', 0.020),
    ('h', 0.061),
    ('i', 0.070),
    ('j', 0.002),
    ('k', 0.008),
    ('l', 0.040),
    ('m', 0.024),
    ('n', 0.067),
    ('o', 0.075),
    ('p', 0.019),
    ('q', 0.001),
    ('r', 0.060),
    ('s', 0.063),
    ('t', 0.091),
    ('u', 0.028),
    ('v', 0.010),
    ('w', 0.024),
    ('x', 0.002),
    ('y', 0.020),
    ('z', 0.001)
]

In [166]:
alpha_analysis

[('a', 0.082),
 ('b', 0.015),
 ('c', 0.028),
 ('d', 0.043),
 ('e', 0.127),
 ('f', 0.022),
 ('g', 0.02),
 ('h', 0.061),
 ('i', 0.07),
 ('j', 0.002),
 ('k', 0.008),
 ('l', 0.04),
 ('m', 0.024),
 ('n', 0.067),
 ('o', 0.075),
 ('p', 0.019),
 ('q', 0.001),
 ('r', 0.06),
 ('s', 0.063),
 ('t', 0.091),
 ('u', 0.028),
 ('v', 0.01),
 ('w', 0.024),
 ('x', 0.002),
 ('y', 0.02),
 ('z', 0.001)]

In [167]:
def preprocess_input_text(text, alphabet):
    new_text = ''
    for c in text:
        if c not in alphabet:
            continue
        new_text += c
    return new_text

In [168]:
input_text = """The first well-documented description of a polyalphabetic cipher was by Leon Battista Alberti around 1467 and used a metal cipher disk to switch between cipher alphabets. Alberti's system only switched alphabets after several words, and switches were indicated by writing the letter of the corresponding alphabet in the ciphertext. Later, Johannes Trithemius, in his work Polygraphiae (which was completed in manuscript form in 1508 but first published in 1518),[5] invented the tabula recta, a critical component of the Vigenère cipher.[6] The Trithemius cipher, however, provided a progressive, rather rigid and predictable system for switching between cipher alphabets.[note 1]
In 1586 Blaise de Vigenère published a type of polyalphabetic cipher called an autokey cipher – because its key is based on the original plaintext – before the court of Henry III of France.[7] The cipher now known as the Vigenère cipher, however, is that originally described by Giovan Battista Bellaso in his 1553 book La cifra del Sig. Giovan Battista Bellaso.[8] He built upon the tabula recta of Trithemius but added a repeating "countersign" (a key) to switch cipher alphabets every letter. Whereas Alberti and Trithemius used a fixed pattern of substitutions, Bellaso's scheme meant the pattern of substitutions could be easily changed, simply by selecting a new key. Keys were typically single words or short phrases, known to both parties in advance, or transmitted "out of band" along with the message. Bellaso's method thus required strong security for only the key. As it is relatively easy to secure a short key phrase, such as by a previous private conversation, Bellaso's system was considerably more secure.[citation needed]
In the 19th century, the invention of Bellaso's cipher was misattributed to Vigenère. David Kahn, in his book, The Codebreakers lamented this misattribution, saying that history had "ignored this important contribution and instead named a regressive and elementary cipher for him [Vigenère] though he had nothing to do with it".[9]
The Vigenère cipher gained a reputation for being exceptionally strong. Noted author and mathematician Charles Lutwidge Dodgson (Lewis Carroll) called the Vigenère cipher unbreakable in his 1868 piece "The Alphabet Cipher" in a children's magazine. In 1917, Scientific American described the Vigenère cipher as "impossible of translation".[10][11] That reputation was not deserved. Charles Babbage is known to have broken a variant of the cipher as early as 1854 but did not publish his work.[12] Kasiski entirely broke the cipher and published the technique in the 19th century, but even in the 16th century, some skilled cryptanalysts could occasionally break the cipher.[9]
Cryptographic slide rule used as a calculation aid by the Swiss Army between 1914 and 1940.The Vigenère cipher is simple enough to be a field cipher if it is used in conjunction with cipher disks.[13] The Confederate States of America, for example, used a brass cipher disk to implement the Vigenère cipher during the American Civil War. The Confederacy's messages were far from secret, and the Union regularly cracked its messages. Throughout the war, the Confederate leadership primarily relied upon three key phrases: "Manchester Bluff", "Complete Victory" and, as the war came to a close, "Come Retribution".[14]
A Vernam cipher whose key is as long as the message becomes a one-time pad, a theoretically unbreakable cipher.[15] Gilbert Vernam tried to repair the broken cipher (creating the Vernam–Vigenère cipher in 1918), but the technology he used was so cumbersome as to be impracticable.[16]"""

In [169]:
text = preprocess_input_text(input_text.lower(), alphabet)
text

'thefirstwelldocumenteddescriptionofapolyalphabeticcipherwasbyleonbattistaalbertiaroundandusedametalcipherdisktoswitchbetweencipheralphabetsalbertissystemonlyswitchedalphabetsafterseveralwordsandswitcheswereindicatedbywritingtheletterofthecorrespondingalphabetintheciphertextlaterjohannestrithemiusinhisworkpolygraphiaewhichwascompletedinmanuscriptforminbutfirstpublishedininventedthetabularectaacriticalcomponentofthevigenrecipherthetrithemiuscipherhoweverprovidedaprogressiveratherrigidandpredictablesystemforswitchingbetweencipheralphabetsnoteinblaisedevigenrepublishedatypeofpolyalphabeticciphercalledanautokeycipherbecauseitskeyisbasedontheoriginalplaintextbeforethecourtofhenryiiioffrancetheciphernowknownasthevigenrecipherhoweveristhatoriginallydescribedbygiovanbattistabellasoinhisbooklacifradelsiggiovanbattistabellasohebuiltuponthetabularectaoftrithemiusbutaddedarepeatingcountersignakeytoswitchcipheralphabetseveryletterwhereasalbertiandtrithemiususedafixedpatternofsubstitutionsbellasossc

In [170]:
encoded_text = encode_vigenere("crypto", text, alphabet)
encoded_text

'vycubfukutezffajfspkcswsutpxihkfldyorfjntzryyqxhktaxivgiuplpaccdgpckrxlhcrjqxfvzyghipuycwiuvbpfsvrjrbdjvpsbgmkmhpwvtfqxhyvccvwrycgtzryyqxhurjqxfvzqhrgvvkdgzajuxmqjvbpedjrztmgcwrtkggmcgtzyfpslopuqlbheychpstvgcwwerrtwpanpxmwpxrwxzgkrtkchkftvctichicpugczongfpusvzliaseznwxfvvvieovvpyhvceltlhtzrwxaklqxgvkjudkyrfjnzfcgfxtsyygrakcjadfdnvrtwwpdycngeigemtqikxgpwkdxkgvgsqewuycsbbkettghgurwxhcssatfgtrptqtzrxvontmbicpvlihtvyckbugeptvwrycgmvgkpxmvgdgjlqkgftkvqnckxfrimkbrguyekciichlwxvppmvgipxzwfrlsifgugrmodcchrgvvkuhfungivvkeeqxhyvccvwrycgtzryyqxhuemixwpsjpbgguckbugeptiidcghasfrrnisqwndemccnwtpgkgrvwrycgvonccstbclrddsatgeastscrtiuvgilygpghuouvbdghjvmgbukeyaizczlixlvscuhfgkftvcwirdyvgepnbwkfdukoptciaseznwxfpfuzgcyeyhmvgmgvxbtvaxivgifdpsxvpxlhjrrdkwizlpezauchvfkscsumizmktbdrribgvrztezcjmxgvkjzdhynraxyfcucalwixgdoopsyimwukyqxznrqdasdlgamirfliasvrzjeotvaitchkpxmvgdgjlpwkyswsfrptisckgczqqllixfuzectygprdlkkkawvwrycgtzryyqxhuvttkmnvrixfyycgxourjqxfvzycwhtzrwxaklqjlsfrdxqsfgyimstemulidjrxmivzmclpgcjplcuja

In [171]:
# count_ngrams(gen_ngrams(encoded_text))

In [172]:
# get_gcdc_ngrams(encoded_text)

In [173]:
# get_keys_length(encoded_text)

In [174]:
get_top_key_length(encoded_text)

[(6, 82),
 (2, 9),
 (1, 8),
 (3, 6),
 (66, 6),
 (12, 5),
 (18, 5),
 (30, 3),
 (4, 2),
 (54, 2),
 (24, 2),
 (42, 2),
 (48, 1),
 (108, 1),
 (9, 1),
 (36, 1),
 (60, 1),
 (10, 1)]

In [202]:
def get_freqs_by_positions(text, alpha_analysis, n = 3):
    top_keys_length = get_top_key_length(text)[0][0]
    
    pos_freqs = dict()
    for pos in range(0, top_keys_length):
        letters_freqs = dict()
        total_sum = 0
        for i in range(pos, len(text), top_keys_length):
            c = text[i]
            if c not in letters_freqs:
                letters_freqs[c] = 0
            letters_freqs[c] += 1
            total_sum += 1
            
        for c in alphabet:
            is_present = False
            for item in letters_freqs:
                if item[0] == c:
                    is_present = True
                    break
            if not is_present:
                letters_freqs[c] = 0.0
        
        letters_freqs = [(item[0], item[1] / total_sum) for item in sorted(letters_freqs.items(), key=lambda item: item[0])]
        pos_freqs[pos] = letters_freqs
    return pos_freqs


def match_dicts_with_alpha(text, alpha_analysis, n = 3):
    pos_freqss = get_freqs_by_positions(text, alpha_analysis)
    matches = dict()
    for pos, letters_to_match in pos_freqss.items():
        alpha_to_text = dict()
        best_offset = find_min_offset(alpha_analysis, letters_to_match)
        matches[pos] = [letters_to_match[i % len(letters_to_match)] for i in range(best_offset, len(letters_to_match) + best_offset)]
    return matches


def find_min_offset(alpha_freqs, pos_freqs):
    n = min(len(alpha_freqs), len(pos_freqs))
    best_offset = 0
    best_min_delta = 1
    for offset in range(0, n):
        delta = 0
        for i in range(0, n):
            delta += ((alpha_freqs[i][1] - pos_freqs[(i + offset) % n][1]) ** 2) / n
        if delta < best_min_delta:
            best_min_delta = delta
            best_offset = offset
    return best_offset

In [203]:
pos_freqs = get_freqs_by_positions(encoded_text, alpha_analysis)

In [204]:
for k, d in pos_freqs.items():
    print(f'{k}:')
    print(d)

0:
[('a', 0.02510460251046025), ('b', 0.0), ('c', 0.06694560669456066), ('d', 0.027196652719665274), ('e', 0.03347280334728033), ('f', 0.04184100418410042), ('g', 0.13389121338912133), ('h', 0.010460251046025104), ('i', 0.016736401673640166), ('j', 0.058577405857740586), ('k', 0.07112970711297072), ('l', 0.0), ('m', 0.008368200836820083), ('n', 0.03556485355648536), ('o', 0.006276150627615063), ('p', 0.06485355648535565), ('q', 0.043933054393305436), ('r', 0.04602510460251046), ('s', 0.0), ('t', 0.06276150627615062), ('u', 0.08368200836820083), ('v', 0.10460251046025104), ('w', 0.029288702928870293), ('x', 0.010460251046025104), ('y', 0.016736401673640166), ('z', 0.0020920502092050207)]
1:
[('a', 0.0), ('b', 0.008385744234800839), ('c', 0.03563941299790356), ('d', 0.027253668763102725), ('e', 0.050314465408805034), ('f', 0.04821802935010482), ('g', 0.031446540880503145), ('h', 0.0020964360587002098), ('i', 0.08385744234800839), ('j', 0.050314465408805034), ('k', 0.09014675052410902), (

In [205]:
dict_with_answers = match_dicts_with_alpha(encoded_text, alpha_analysis)

In [213]:
dict_with_answers

{0: [('c', 0.06694560669456066),
  ('d', 0.027196652719665274),
  ('e', 0.03347280334728033),
  ('f', 0.04184100418410042),
  ('g', 0.13389121338912133),
  ('h', 0.010460251046025104),
  ('i', 0.016736401673640166),
  ('j', 0.058577405857740586),
  ('k', 0.07112970711297072),
  ('l', 0.0),
  ('m', 0.008368200836820083),
  ('n', 0.03556485355648536),
  ('o', 0.006276150627615063),
  ('p', 0.06485355648535565),
  ('q', 0.043933054393305436),
  ('r', 0.04602510460251046),
  ('s', 0.0),
  ('t', 0.06276150627615062),
  ('u', 0.08368200836820083),
  ('v', 0.10460251046025104),
  ('w', 0.029288702928870293),
  ('x', 0.010460251046025104),
  ('y', 0.016736401673640166),
  ('z', 0.0020920502092050207),
  ('a', 0.02510460251046025),
  ('b', 0.0)],
 1: [('r', 0.08385744234800839),
  ('s', 0.029350104821802937),
  ('t', 0.050314465408805034),
  ('u', 0.033542976939203356),
  ('v', 0.13417190775681342),
  ('w', 0.008385744234800839),
  ('x', 0.020964360587002098),
  ('y', 0.06918238993710692),
  ('

In [210]:
for pos, alphas_offsets in dict_with_answers.items():
    print(f"{pos} {alphas_offsets[0][0]}")

0 c
1 r
2 y
3 p
4 t
5 o


In [214]:
decode_vigenere("crypto", encoded_text, alphabet)

'thefirstwelldocumenteddescriptionofapolyalphabeticcipherwasbyleonbattistaalbertiaroundandusedametalcipherdisktoswitchbetweencipheralphabetsalbertissystemonlyswitchedalphabetsafterseveralwordsandswitcheswereindicatedbywritingtheletterofthecorrespondingalphabetintheciphertextlaterjohannestrithemiusinhisworkpolygraphiaewhichwascompletedinmanuscriptforminbutfirstpublishedininventedthetabularectaacriticalcomponentofthevigenrecipherthetrithemiuscipherhoweverprovidedaprogressiveratherrigidandpredictablesystemforswitchingbetweencipheralphabetsnoteinblaisedevigenrepublishedatypeofpolyalphabeticciphercalledanautokeycipherbecauseitskeyisbasedontheoriginalplaintextbeforethecourtofhenryiiioffrancetheciphernowknownasthevigenrecipherhoweveristhatoriginallydescribedbygiovanbattistabellasoinhisbooklacifradelsiggiovanbattistabellasohebuiltuponthetabularectaoftrithemiusbutaddedarepeatingcountersignakeytoswitchcipheralphabetseveryletterwhereasalbertiandtrithemiususedafixedpatternofsubstitutionsbellasossc